# Diplomatura en Ciencia de Datos, Aprendizaje Automático y sus Aplicaciones

# Exploración y Curación de Datos

Edición 2022

En esta notebook, vamos a cargar el conjunto de datos de la compentencia Kaggle sobre estimación de precios de ventas de propiedades en Melbourne, Australia.

Utilizaremos el conjunto de datos reducido producido por DanB. Hemos subido una copia a un servidor de la Universidad Nacional de Córdoba para facilitar su acceso remoto.

In [1]:
# Import the neccesary libraries

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

import seaborn as sns
sns.set_context('talk')

from decouple import config
from sqlalchemy import create_engine, text

In [2]:
import plotly
plotly.__version__
# Make sure it's 4.14.3

'5.8.0'

In [3]:
kaggle_df = pd.read_csv(
    'https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/melb_data.csv'
)

kaggle_df[:5]

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0
3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,...,2.0,1.0,94.0,NaN,NaN,Yarra,-37.7969,144.9969,Northern Metropolitan,4019.0
4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,...,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.8072,144.9941,Northern Metropolitan,4019.0


In [4]:
kaggle_df.shape

(13580, 21)

In [5]:
kaggle_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Suburb         13580 non-null  object 
 1   Address        13580 non-null  object 
 2   Rooms          13580 non-null  int64  
 3   Type           13580 non-null  object 
 4   Price          13580 non-null  float64
 5   Method         13580 non-null  object 
 6   SellerG        13580 non-null  object 
 7   Date           13580 non-null  object 
 8   Distance       13580 non-null  float64
 9   Postcode       13580 non-null  float64
 10  Bedroom2       13580 non-null  float64
 11  Bathroom       13580 non-null  float64
 12  Car            13518 non-null  float64
 13  Landsize       13580 non-null  float64
 14  BuildingArea   7130 non-null   float64
 15  YearBuilt      8205 non-null   float64
 16  CouncilArea    12211 non-null  object 
 17  Lattitude      13580 non-null  float64
 18  Longti

In [6]:
kaggle_df.describe()

,Rooms,Price,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
count,13580.000000,1.358000e+04,13580.000000,13580.000000,13580.000000,13580.000000,13518.000000,13580.000000,7130.000000,8205.000000,13580.000000,13580.000000,13580.000000
mean,2.937997,1.075684e+06,10.137776,3105.301915,2.914728,1.534242,1.610075,558.416127,151.967650,1964.684217,-37.809203,144.995216,7454.417378
std,0.955748,6.393107e+05,5.868725,90.676964,0.965921,0.691712,0.962634,3990.669241,541.014538,37.273762,0.079260,0.103916,4378.581772
min,1.000000,8.500000e+04,0.000000,3000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1196.000000,-38.182550,144.431810,249.000000
25%,2.000000,6.500000e+05,6.100000,3044.000000,2.000000,1.000000,1.000000,177.000000,93.000000,1940.000000,-37.856822,144.929600,4380.000000
50%,3.000000,9.030000e+05,9.200000,3084.000000,3.000000,1.000000,2.000000,440.000000,126.000000,1970.000000,-37.802355,145.000100,6555.000000
75%,3.000000,1.330000e+06,13.000000,3148.000000,3.000000,2.000000,2.000000,651.000000,174.000000,1999.000000,-37.756400,145.058305,10331.000000
max,10.000000,9.000000e+06,48.100000,3977.000000,20.000000,8.000000,10.000000,433014.000000,44515.000000,2018.000000,-37.408530,145.526350,21650.000000


In [7]:
# view column names
kaggle_df.columns

Index(['Suburb', 'Address', 'Rooms', 'Type', 'Price', 'Method', 'SellerG',
       'Date', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car',
       'Landsize', 'BuildingArea', 'YearBuilt', 'CouncilArea', 'Lattitude',
       'Longtitude', 'Regionname', 'Propertycount'],
      dtype='object')

In [8]:
# check for missing values
kaggle_df.isnull().sum()

Suburb              0
Address             0
Rooms               0
Type                0
Price               0
Method              0
SellerG             0
Date                0
Distance            0
Postcode            0
Bedroom2            0
Bathroom            0
Car                62
Landsize            0
BuildingArea     6450
YearBuilt        5375
CouncilArea      1369
Lattitude           0
Longtitude          0
Regionname          0
Propertycount       0
dtype: int64

# Ejercicio 1 SQL:

1. Crear una base de datos en SQLite utilizando la libreria SQLalchemy. https://docs.sqlalchemy.org/en/14/core/engines.html#sqlite

2. Ingestar los datos provistos en 'https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/melb_data.csv' en una tabla y el dataset generado en clase con datos de airbnb y sus precios por codigo postal en otra.

3. Implementar consultas en SQL que respondan con la siguiente información:

- cantidad de registros totales por ciudad.
- cantidad de registros totales por barrio y ciudad.

4. Combinar los datasets de ambas tablas ingestadas utilizando el comando JOIN de SQL para obtener un resultado similar a lo realizado con Pandas en clase.

In [9]:
dbENGINE = config("DB_ENGINE")
dbNAME = config("DB_NAME")

In [10]:
# Define de Function Class of Database

class ManageDatabase():
    
    def __init__(self, kaggle_df, airbn_df, db_engine, db_name, echo=False):
        
        self.kaggle_df = kaggle_df
        self.airbnb_df = airbnb_df
        self.db_engine = db_engine
        self.db_name = db_name
        self.echo = echo
    
    def create_database(self):
        
        engine = create_engine(
            r"{}:///{}".format(
                self.db_engine,
                os.path.join(
                    os.getcwd(),
                    self.db_name,
                )
            ), 
            echo=self.echo
        )
        
        return engine
    
    def data_ingestion(self):
        '''Data ingestion on property sales price estimation in Melbourne, Australia; by Kaggle.           
           A new database is created to be used with SQL language
        ''' 
        self.kaggle_df.to_sql("kaggle_sql", con=self.create_database(), if_exists="replace")
        self.airbnb_df.to_sql("airbnb_sql", con=self.create_database(), if_exists="replace")        
    
    def queries(self):        
        # Cantidad de registros totales por ciudad en dataset Kaggle
        query1 = "SELECT CouncilArea, COUNT(CouncilArea) FROM kaggle_sql GROUP BY CouncilArea"

        # Cantidad de registros totales por barrio y ciudad en Kaggle
        query2 = "SELECT CouncilArea, Suburb, COUNT(Suburb) FROM kaggle_sql GROUP BY Suburb ORDER BY CouncilArea"
        
        return [query1, query2]

In [11]:
# Ingesta de datos de conjunto Airbnb
interesting_cols = [
  'description', 'neighborhood_overview',
  'street', 'neighborhood', 'city', 'suburb', 'state', 'zipcode',
  'price', 'weekly_price', 'monthly_price',
  'latitude', 'longitude',
]
airbnb_df = pd.read_csv(
    'https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/cleansed_listings_dec18.csv',
    usecols=interesting_cols,
)

C:\Users\pablonicolasr\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (35) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [12]:
echo = True
database = ManageDatabase(kaggle_df, airbnb_df, dbENGINE, dbNAME, echo)

In [13]:
engine_create = database.create_database() 

In [14]:
database.data_ingestion() 

2022-05-18 14:44:17,756 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("kaggle_sql")
2022-05-18 14:44:17,757 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-05-18 14:44:17,758 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("kaggle_sql")
2022-05-18 14:44:17,759 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-05-18 14:44:17,763 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-05-18 14:44:17,765 INFO sqlalchemy.engine.Engine 
CREATE TABLE kaggle_sql (
	"index" BIGINT, 
	"Suburb" TEXT, 
	"Address" TEXT, 
	"Rooms" BIGINT, 
	"Type" TEXT, 
	"Price" FLOAT, 
	"Method" TEXT, 
	"SellerG" TEXT, 
	"Date" TEXT, 
	"Distance" FLOAT, 
	"Postcode" FLOAT, 
	"Bedroom2" FLOAT, 
	"Bathroom" FLOAT, 
	"Car" FLOAT, 
	"Landsize" FLOAT, 
	"BuildingArea" FLOAT, 
	"YearBuilt" FLOAT, 
	"CouncilArea" TEXT, 
	"Lattitude" FLOAT, 
	"Longtitude" FLOAT, 
	"Regionname" TEXT, 
	"Propertycount" FLOAT
)


2022-05-18 14:44:17,766 INFO sqlalchemy.engine.Engine [no key 0.00099s] ()
2022-05-18 14:44:18,004 INFO sq

2022-05-18 14:44:19,623 INFO sqlalchemy.engine.Engine COMMIT


In [15]:
with engine_create.connect() as con:
    try:
        
        for query in database.data_ingestion():

            rs = con.execute(query)

            print(query)

            for row in rs:

                print(row)

            print("\n\n")
            
    except Exception as e:
            
            print(e)
            
            print("\n\n")      
    

2022-05-18 14:44:30,041 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("kaggle_sql")
2022-05-18 14:44:30,042 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-05-18 14:44:30,045 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("kaggle_sql")
2022-05-18 14:44:30,045 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-05-18 14:44:30,047 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2022-05-18 14:44:30,048 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-05-18 14:44:30,050 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("kaggle_sql")
2022-05-18 14:44:30,051 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-05-18 14:44:30,056 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2022-05-18 14:44:30,056 INFO sqlalchemy.engine.Engine [raw sql] ('kaggle_sql',)
2022-05-18 14:44:30,059 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_key_li

2022-05-18 14:44:31,214 INFO sqlalchemy.engine.Engine PRAGMA main.index_info("ix_airbnb_sql_index")
2022-05-18 14:44:31,215 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-05-18 14:44:31,217 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2022-05-18 14:44:31,218 INFO sqlalchemy.engine.Engine [raw sql] ('airbnb_sql',)
2022-05-18 14:44:31,220 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-05-18 14:44:31,220 INFO sqlalchemy.engine.Engine 
DROP TABLE airbnb_sql
2022-05-18 14:44:31,221 INFO sqlalchemy.engine.Engine [no key 0.00062s] ()
2022-05-18 14:44:31,439 INFO sqlalchemy.engine.Engine COMMIT
2022-05-18 14:44:31,447 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-05-18 14:44:31,448 INFO sqlalchemy.engine.Engine 
CREATE TABLE airbnb_sql (
	"index" BIGINT, 
	description TEXT, 
	neighborhood_overview TEXT, 
	street TEXT, 
	neighborhood TEXT, 
	city TEXT, 
	suburb TEXT, 
	stat

2022-05-18 14:44:32,288 INFO sqlalchemy.engine.Engine COMMIT
'NoneType' object is not iterable



